# Copernicus Data to Indicators

Processing of the Copernicus data into census output areas

In [1]:
# libraries
# Load R libraries
 if(!require("pacman"))
     install.packages("pacman")
     library("pacman")

p_load("dplyr", "sf", "purrr", "tidyverse", "terra", "raster", "exactextractr")

print("Loaded Packages:")
p_loaded()

Loading required package: pacman



[1] "Loaded Packages:"


[1] "exactextractr" "raster"        "sp"            "terra"        
 [5] "lubridate"     "forcats"       "stringr"       "readr"        
 [9] "tidyr"         "tibble"        "ggplot2"       "tidyverse"    
[13] "purrr"         "sf"            "dplyr"         "pacman"

In [2]:
# create the pipeline directory if it does not exist
pipelineDir <- file.path("../..","2_pipeline","Athens","1b_Copernicus","2011")
if(!dir.exists(pipelineDir)){
    dir.create(pipelineDir, recursive = TRUE)
    print(paste0(pipelineDir, " created"))
}

country <- "Greece"

In [3]:
# Setup

# Copernicus folder
copDir <- "../../0_data/Athens/Copernicus/"

highResLayers <- c("IMD", "TCD") # testing

# census output areas
censusOA <- st_read("../../0_data/Athens/OA/GR.Municipalities.shp")

#terraOptions(tempdir = pipelineDir)

Reading layer `GR.Municipalities' from data source 
  `/Cities/0_data/Athens/OA/GR.Municipalities.shp' using driver `ESRI Shapefile'
Simple feature collection with 325 features and 14 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: 104040.7 ymin: 3850938 xmax: 1007943 ymax: 4624010
Projected CRS: GGRS87 / Greek Grid


In [4]:
# need to join copernicus tiles together
for(cop in highResLayers){
    print(paste0("Processing: ", cop))
    flush.console()
    
    #export path
    countryCopPath <- paste0(pipelineDir, "/", country, "_", cop, ".tif")

    # have the tiles been joined together already?
    countryCopCheck <- file.exists(countryCopPath)
    
    if(countryCopCheck){
        print(paste0(countryCopCheck, " already created"))
        flush.console()
    } else {
        # join the tiles together and export it
        
        # get the files
        tiles <- list.files(copDir, full.names = TRUE, recursive = TRUE, pattern=paste0(".*", cop, ".*\\.tif$"))  
        
        # create an image catalog
        ic <- terra::sprc(lapply(tiles, rast))
        
        # mosic the tiles
        icMosaic <- mosaic(ic, filename = countryCopPath, fun="max", overwrite=TRUE)
        #icMosaic <- merge(ic, filename = countryCopPath, overwrite=TRUE) # quicker
    }
}

[1] "Processing: IMD"
[1] "TRUE already created"
[1] "Processing: TCD"
[1] "TRUE already created"


In [5]:
Sys.time()
censusOA_IMD <- censusOA

# Impervious surface
copRaster <- rast(paste0(pipelineDir, "/", country, "_", "IMD", ".tif"))

# #calculate the area of the raster cell (m2)
cellArea <- res(copRaster)[1]*res(copRaster)[2]

# #calculate the number of cells within each output area  
censusOA_IMD$cellArea_m2 <- exact_extract(copRaster, censusOA_IMD, 'count', coverage_area = TRUE, progress = TRUE)

# calculate the area of each output area that is impervious
censusOA_IMD$impArea_m2 <- exact_extract(copRaster, censusOA_IMD, 'sum', progress = TRUE)

# #calculate the percentage of each output area that is impervious
censusOA_IMD$impPerc <- round((censusOA_IMD$impArea_m2/censusOA_IMD$cellArea_m2)*100, 3)

# head(censusOA_IMD)
Sys.time()

[1] "2025-01-15 21:58:20 GMT"

Warning message in .local(x, y, ...):
“Polygons transformed to raster CRS (EPSG:3035)”
Cannot preload entire working area of 6928191312 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.




 |======================================================================| 100%

Warning message in .local(x, y, ...):
“Polygons transformed to raster CRS (EPSG:3035)”
Cannot preload entire working area of 6928191312 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.




 |======================================================================| 100%

[1] "2025-01-15 22:18:40 GMT"

In [6]:
#output the data as a geojson
st_write(censusOA_IMD, file.path(pipelineDir, "oa_IMD.geojson"), delete_dsn = TRUE)

Deleting source `../../2_pipeline/Athens/1b_Copernicus/2011/oa_IMD.geojson' using driver `GeoJSON'
Writing layer `oa_IMD' to data source 
  `../../2_pipeline/Athens/1b_Copernicus/2011/oa_IMD.geojson' using driver `GeoJSON'
Writing 325 features with 17 fields and geometry type Multi Polygon.


In [7]:
Sys.time()
censusOA_TCD <- censusOA
# TREE COVER DENSITY
# cell size = 100 m2, therefore a percentage value directly equals m2, i.e. 100% coverage = 100 m2 area of impervious surface

# data
copRaster <- rast(paste0(pipelineDir, "/", country, "_", "TCD", ".tif"))

# #calculate the area of the raster cell (m2)
cellArea <- res(copRaster)[1]*res(copRaster)[2]

# #calculate the number of cells within each output area 
censusOA_TCD$cellArea_m2 <- exact_extract(copRaster, censusOA_TCD, 'count', coverage_area = TRUE, progress = TRUE)

# calculate the area of each output area that has tree cover
censusOA_TCD$tcdArea_m2 <- exact_extract(copRaster, censusOA_TCD, 'sum', progress = TRUE)

# #calculate the percentage of each output area that has tree cover
censusOA_TCD$tcdPerc <- round((censusOA_TCD$tcdArea_m2/censusOA_TCD$cellArea_m2)*100, 3)

# head(censusOA_TCD)
Sys.time()

[1] "2025-01-15 22:18:42 GMT"

Warning message in .local(x, y, ...):
“Polygons transformed to raster CRS (EPSG:3035)”
Cannot preload entire working area of 6928191312 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.




 |======================================================================| 100%

Warning message in .local(x, y, ...):
“Polygons transformed to raster CRS (EPSG:3035)”
Cannot preload entire working area of 6928191312 cells with max_cells_in_memory = 3e+07. Raster values will be read for each feature individually.




 |======================================================================| 100%

[1] "2025-01-15 22:44:12 GMT"

In [8]:
#output the data as a geojson
st_write(censusOA_TCD, file.path(pipelineDir, "oa_TCD.geojson"), delete_dsn = TRUE)

Deleting source `../../2_pipeline/Athens/1b_Copernicus/2011/oa_TCD.geojson' using driver `GeoJSON'
Writing layer `oa_TCD' to data source 
  `../../2_pipeline/Athens/1b_Copernicus/2011/oa_TCD.geojson' using driver `GeoJSON'
Writing 325 features with 17 fields and geometry type Multi Polygon.


**END**